In [143]:
import requests
import json
import pandas as pd
from lxml import html


base_url = 'http://www.data.jma.go.jp/gmd/risk/obsdl/index.php'

url = 'http://www.data.jma.go.jp/gmd/risk/obsdl/show/table.html'
first_resp = requests.get(base_url)

PHPSESSID = html.fromstring(first_resp.text).xpath('//input[@id="sid"]')[0].get('value') 

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Host': 'www.data.jma.go.jp',
    'Origin': 'http://www.data.jma.go.jp',
    'Referer': 'http://www.data.jma.go.jp/gmd/risk/obsdl/index.php',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}

payload_dic = {
    'stationNumList': '["s47784"]',
    'aggrgPeriod': '9',
    'elementNumList': '[["201",""],["101",""],["503",""],["501",""],["401",""],["301",""],["610",""],["605",""],["612",""],["703",""],["607",""],["704",""]]',
    'interAnnualFlag': '1',
    'ymdList': '["2020","2020","1","1","1","3"]',
    'optionNumList': '[]',
    'rmkFlag': '1',
    'disconnectFlag': '1',
    'kijiFlag': '0',
    'huukouFlag': '0',
    'youbiFlag': '0',
    'fukenFlag': '0',
    'jikantaiFlag': '0',
    'jikantaiList': '[1, 24]',
    'PHPSESSID': PHPSESSID
}

response = requests.post(url, headers = headers, data=payload_dic)
print(response.text)
data = json.loads(response.text)
header = data['header']

{"width":880,"height":500,"header":[{"id":"period","name":"<div class='header-cell-noborder'>\u5e74\u6708\u65e5<\/div>","field":"period","width":160},{"id":"id0_0_0","name":"<div class='header-cell'>\u5c71\u53e3<\/div><div class='header-cell-noborder'>\u6c17\u6e29(\u2103)<\/div>","field":"data0_0_0","width":80},{"id":"id0_1_0","name":"<div class='header-cell'>\u5c71\u53e3<\/div><div class='header-cell-noborder'>\u964d\u6c34\u91cf(mm)<\/div>","field":"data0_1_0","width":112},{"id":"id0_2_0","name":"<div class='header-cell'>\u5c71\u53e3<\/div><div class='header-cell-noborder'>\u964d\u96ea(cm)<\/div>","field":"data0_2_0","width":96},{"id":"id0_3_0","name":"<div class='header-cell'>\u5c71\u53e3<\/div><div class='header-cell-noborder'>\u7a4d\u96ea(cm)<\/div>","field":"data0_3_0","width":96},{"id":"id0_4_0","name":"<div class='header-cell'>\u5c71\u53e3<\/div><div class='header-cell-noborder'>\u65e5\u7167\u6642\u9593(\u6642\u9593)<\/div>","field":"data0_4_0","width":128},{"id":"id0_5_0","name

In [268]:
# header = [html.fromstring(el['name']).xpath('//div[@class="header-cell-noborder"]')[0].text for el in data['header']]
import re
import os
from google.cloud import translate_v2

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\DEV\GoogleTranslationAPI\GoogleCloudKey_AdilbekServiceAccount.json"
translate_client = translate_v2.Client()
target_language = 'en'


def translate_word(row, target_language):
    transl_row = []
    for word in row: 
        if word is not None and word!='--' and not re.match(r"^[-+]?\d*\.\d+$|^\d+$", word):
            transl_word = translate_client.translate(
                        word,
                        target_language=target_language
                        )["translatedText"]
        else:
            transl_word = word
            
        transl_row.append(transl_word)
#     print(transl_word)
    return transl_row


header = ['Precipitation(mm)', 'Temperature_avg', 'Dew', 'SteamPressure(hPa)',

          'Humidity_avg', 'WindSpeed(m/s)', 'Wind_direction',

          'Sunshine(h)', 'Total_solarradiation(MJ/m2)', 'Snowfall(cm)',

          'Deepsnowfall(cm)', 'Weather', 'Cloudamount', 'Visibility(km)']
print(header)
df = pd.DataFrame()

body = data['data']
for row in body:
    row_data = []
    for cell in row:
        cell_html = html.fromstring(row[cell])
        images = cell_html.xpath('//img')
        if len(images) == 0:
            row_data.append(cell_html.text)
        else:
            for img in images:
                row_data.append(img.get('alt'))
                
    print(row_data)
    transl_row_data = translate_word(row_data, target_language)
    print(transl_row_data)
    
#     print(transl_row_data)
#     print([type(el) for el in row_data])
    
    
    df = df.append([transl_row_data], ignore_index=True)
    print('****************************************')

df.columns = header
print(df)

['Precipitation(mm)', 'Temperature_avg', 'Dew', 'SteamPressure(hPa)', 'Humidity_avg', 'WindSpeed(m/s)', 'Wind_direction', 'Sunshine(h)', 'Total_solarradiation(MJ/m2)', 'Snowfall(cm)', 'Deepsnowfall(cm)', 'Weather', 'Cloudamount', 'Visibility(km)']
['2020年1月1日1時', '2.6', '--', '--', '--', '--', '1.8', '西北西', None, '70', '-2.3', '晴れ', None, '20.0']
['January 1, 2020 1:00', '2.6', '--', '--', '--', '--', '1.8', 'West-northwest', None, '70', '-2.3', 'Sunny', None, '20.0']
****************************************
['2020年1月1日2時', '1.0', '--', '--', '--', '--', '0.5', '南西', None, '80', '-2.1', '晴れ', None, '20.0']
['January 1, 2020 at 2 o&#39;clock', '1.0', '--', '--', '--', '--', '0.5', 'Southwest', None, '80', '-2.1', 'Sunny', None, '20.0']
****************************************
['2020年1月1日3時', '0.1', '--', '--', '--', '--', '1.3', '北東', None, '89', '-1.5', '晴れ', None, '20.0']
['January 1, 2020 at 3 o&#39;clock', '0.1', '--', '--', '--', '--', '1.3', 'Northeast', None, '89', '-1.5', 'Sunny

['January 2, 2020, 7 o&#39;clock', '1.6', '--', '--', '--', '--', '1.3', 'Northeast', None, '93', '0.6', 'Tan', None, '20.0']
****************************************
['2020年1月2日8時', '1.9', '--', '--', '--', '0.0', '1.2', '東北東', None, '93', '0.9', '晴れ', None, '20.0']
['January 2, 2020, 8 o&#39;clock', '1.9', '--', '--', '--', '0.0', '1.2', 'East northeast', None, '93', '0.9', 'Sunny', None, '20.0']
****************************************
['2020年1月2日9時', '3.2', '--', '--', '--', '0.4', '0.9', '北東', None, '88', '1.4', '晴れ', None, '20.0']
['January 2, 2020, 9 o&#39;clock', '3.2', '--', '--', '--', '0.4', '0.9', 'Northeast', None, '88', '1.4', 'Sunny', None, '20.0']
****************************************
['2020年1月2日10時', '5.7', '--', '--', '--', '0.6', '0.7', '東', None, '80', '2.5', '晴れ', None, '20.0']
['January 2, 2020 at 10:00', '5.7', '--', '--', '--', '0.6', '0.7', 'east', None, '80', '2.5', 'Sunny', None, '20.0']
****************************************
['2020年1月2日11時', '8.9', '--'

['14:00 on January 3, 2020', '12.7', '--', '--', '--', '1.0', '4.6', 'West-northwest', None, '45', '1.1', 'Sunny', None, '20.0']
****************************************
['2020年1月3日15時', '13.1', '--', '--', '--', '1.0', '3.6', '西南西', None, '48', '2.4', '晴れ', None, '19.4']
['15:00 on January 3, 2020', '13.1', '--', '--', '--', '1.0', '3.6', 'Southwest West', None, '48', '2.4', 'Sunny', None, '19.4']
****************************************
['2020年1月3日16時', '12.2', '--', '--', '--', '1.0', '3.9', '西', None, '49', '1.8', '晴れ', None, '19.6']
['16:00, January 3, 2020', '12.2', '--', '--', '--', '1.0', '3.9', 'oo', None, '49', '1.8', 'Sunny', None, '19.6']
****************************************
['2020年1月3日17時', '10.7', '--', '--', '--', '0.7', '2.6', '西', None, '57', '2.5', '晴れ', None, '18.5']
['January 3, 2020 at 17:00', '10.7', '--', '--', '--', '0.7', '2.6', 'oo', None, '57', '2.5', 'Sunny', None, '18.5']
****************************************
['2020年1月3日18時', '8.5', '--', '--', '--',

In [ ]:
import os
from google.cloud import translate_v2

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\DEV\GoogleTranslationAPI\GoogleCloudKey_AdilbekServiceAccount.json"
translate_client = translate_v2.Client()
target_language = 'en'

test_text = '2.6'

output_text = translate_client.translate(
                        test_text,
                        target_language=target_language
                        )["translatedText"]

print(output_text)

In [247]:
import re
lst = ['2020年1月1日3時', '0.1', '--', '--', '--', '--', '1.3', '北東', None, '89', '-1.5', '晴れ', None, '20.0']

for a in lst: 
    if a is not None and a!='--' and not re.match(r"^[-+]?\d*\.\d+$|^\d+$", a):
        print('Translate')
    else:
        print('Not')

Translate
Not
Not
Not
Not
Not
Not
Translate
Not
Not
Not
Translate
Not
Not


In [250]:
import os
from google.cloud import translate_v2

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\DEV\GoogleTranslationAPI\GoogleCloudKey_AdilbekServiceAccount.json"
translate_client = translate_v2.Client()


def translate_word(word, target_language):
    transl_word = translate_client.translate(
                        word,
                        target_language=target_language
                        )["translatedText"]
    
    return transl_word

header = [html.fromstring(el['name']).xpath('//div[@class="header-cell-noborder"]')[0].text for el in data['header']]
transl_header = [translate_word(col, 'en') for col in header]
print(transl_header)
# df = pd.DataFrame()

# body = data['data']
# for row in body:
#     row_data = []
#     for cell in row:
#         cell_html = html.fromstring(row[cell])
#         images = cell_html.xpath('//img')
#         if len(images) == 0:
#             row_data.append(cell_html.text)
#         else:
#             for img in images:
#                 row_data.append(img.get('alt'))
#     print(row_data)
#     print([type(el) for el in row_data])
#     df = df.append([row_data], ignore_index=True)
#     print('****************************************')

# df.columns = header
# print(df)

['date', 'Temperature (℃)', 'Precipitation (mm)', 'Snowfall (cm)', 'Snow (cm)', 'Daylight hours (hours)', 'Wind speed (m/s)', 'wind direction', 'Insolation (MJ/㎡)', 'Relative humidity (%)', 'Dew point temperature (℃)', 'weather', 'Cloud cover (10 points ratio)', 'Visibility (km)']
